In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import interact, IntSlider

#### Simple paramters and data points

In [2]:
n = 4
m = 1000
x = np.array([-1/2, -1/6, 1/6, 1/2])
y = np.array([1/4, 1/30, 1/30, 1/4])

#### Helper Functions

In [3]:
def relu(x):
    return np.maximum(0, x)

def indicator(condition):
    return condition.astype(int)

#### 2-layer ReLU neural network

$ f_m(x;\theta) = \frac{1}{\alpha_m} \sum_{j=1}^m a_j(w_j x + b_j)_{+}$

In [4]:
# 2-Lyaer ReLu Neural Network
def twoLayerReluNet(alpha_m, a, w, b, x):
    return 1/alpha_m * np.sum(a * relu(w * x + b))

#### Initialisation of weights

$a_j^0 \sim \mathcal{N}(0,\beta^2_{1m})$ and $b_j^0, w_j^0 \sim \mathcal{N}(0,\beta^2_{2m})$

through normalisation we define

$\kappa := \frac{\beta_{1m} \beta_{2m}}{\alpha_m}$, $\kappa' := \frac{\beta_{1m}}{\beta_{2m}}$

and

$\gamma := \lim_{m \rightarrow \infty} \frac{\log{\kappa}}{\log{m}}$, $\gamma' := \lim_{m \rightarrow \infty} \frac{\log{\kappa'}}{\log{m}}$

We take that parameters $a_m, \beta_{1m}, \beta_{2m}$ are taken to have a power-law relation to $m$ and so we are able to choose $\gamma$ and $\gamma'$ such that the $a$-lag training regime is used.

<small>  T. Luo, Z.-Q. J. Xu, Z. Ma, and Y. Zhang. Phase diagram for two-layer relu neu
ral networks at innite-width limit . In: Journal of Machine Learning Research 22.71
 (2021), pp. 147.</small>

<small>  Z.Chen,Y.Li,T.Luo,Z.Zhou,andZ.-Q. J. Xu. Phase diagram of initial condensation
 for two-layer neural networks . In: arXiv preprint arXiv:2303.06561 (2023).</small>

In [5]:
## We use the following to fix the values of gamma, gamma_prime and alpha_m
gamma = 3/2
gamma_prime = -1/2

alpha_m = m**(gamma - gamma_prime)
kappa = m**(-gamma)
kappa_prime = m**(-gamma_prime)

beta_1m = math.sqrt(kappa*kappa_prime*alpha_m)
beta_2m = math.sqrt((kappa/kappa_prime)*alpha_m)

#### Gradient flow

Empirical Risk equation - $\mathcal{R}(\theta) := \frac{1}{2n} \sum_{i=1}^m (f_m(x_i; \theta) - y_i)^2$

In [6]:
## Typical parameter gradients
def gradient_flow(a,w,b):
    # Compute wx_plus_b for all i and j
    wx_plus_b = np.outer(x,w) + b  # Shape: (n, m)

    # Apply ReLU activation
    relu_mask = (wx_plus_b > 0).astype(float)  # Shape: (n, m)
    relu = wx_plus_b * relu_mask  # Apply the mask

    # First term inside parentheses: (1 / alpha_m) * sum_j(a_j * ReLU_j)
    term2 = (np.sum(a * relu, axis=1) / alpha_m) - y  # Shape: (n,)

    # Full sum: sum relu multiplied with appropriate term2, over i
    full_sum_a = np.sum(relu * term2[:, np.newaxis], axis=0)  # Shape: (m,)

    # Multiply by aj * xi * 1{wx_plus_b > 0}
    term1_w = (a * x[:, np.newaxis] * relu_mask)  # Shape: (n, m)
    # Full sum: sum term1_w multiplied with appropriate term2, over i
    full_sum_w = np.sum(term1_w * term2[:, np.newaxis], axis=0)  # Shape: (m,)

    # Multiply by aj * 1{wx_plus_b > 0}
    term1_b = (a * relu_mask)  # Shape: (n, m)
    # Full sum: sum term1_b multiplied with appropriate term2, over i
    full_sum_b = np.sum(term1_b * term2[:, np.newaxis], axis=0)  # Shape: (m,)

    # Scale by constants
    grad_a = -full_sum_a / (n * alpha_m)  # Final shape: (m,)
    grad_w = -full_sum_w / (n * alpha_m)  # Final shape: (m,)
    grad_b = -full_sum_b / (n * alpha_m)  # Final shape: (m,)

    return grad_a, grad_w, grad_b

Gradient Descent training

In [7]:
def gradient_descent(step_size, steps):
    
    a = np.random.normal(loc=0, scale=beta_1m, size=math.ceil(m))
    w = np.random.normal(loc=0, scale=beta_2m, size=math.ceil(m))
    b = np.random.normal(loc=0, scale=beta_2m, size=math.ceil(m))

    a_values = []
    w_values = []
    b_values = []

    #for n in range(math.ceil(time/step_size)):
    for n in range(steps): 
        grad_a, grad_w, grad_b = gradient_flow(a,w,b)
        a = a + step_size * grad_a
        w = w + step_size * grad_w
        b = b + step_size * grad_b

        # Save the current values of a, w, and b
        a_values.append(a.copy())
        w_values.append(w.copy())
        b_values.append(b.copy())

        print('Completed step  %d / %d' % (n, steps)) 

    return a_values, w_values, b_values

def gradient_descent_symmetric(step_size, steps):
    
    a = np.random.normal(loc=0, scale=beta_1m, size=math.ceil(m/2))
    w = np.random.normal(loc=0, scale=beta_2m, size=math.ceil(m/2))
    b = np.random.normal(loc=0, scale=beta_2m, size=math.ceil(m/2))

    a = np.concatenate((a, a))
    w = np.concatenate((w, -w))
    b = np.concatenate((b, b))

    a_values = []
    w_values = []
    b_values = []

    #for n in range(math.ceil(time/step_size)):
    for n in range(steps): 
        grad_a, grad_w, grad_b = gradient_flow(a,w,b)
        a = a + step_size * grad_a
        w = w + step_size * grad_w
        b = b + step_size * grad_b

        # Save the current values of a, w, and b
        a_values.append(a.copy())
        w_values.append(w.copy())
        b_values.append(b.copy())

        print('Completed step  %d / %d' % (n, steps)) 

    return a_values, w_values, b_values

#### Training

In [ ]:
# Assuming beta_1m, beta_2m, m, and gradient_flow are defined
n_steps = 200000
a_values, w_values, b_values = gradient_descent(4000, n_steps)

# Convert lists to arrays for easier saving and plotting
a_values = np.array(a_values)
w_values = np.array(w_values)
b_values = np.array(b_values)

# Save the arrays to disk
np.save('weights\\a_values.npy', a_values)
np.save('weights\\w_values.npy', w_values)
np.save('weights\\b_values.npy', b_values)

Code Cell generating function development over time

In [19]:
# Load the arrays from the .npy files
a_values = np.load('weights\\a_values.npy')
w_values = np.load('weights\\w_values.npy')
b_values = np.load('weights\\b_values.npy')

# Define the range of x
x_graph = np.linspace(-0.5, 0.5, 800)
y_graph = np.zeros(len(x_graph))

# Define the function to plot the values at a given index
def plot_values(index):
    a = a_values[index]
    w = w_values[index]
    b = b_values[index]

    # Compute the corresponding y values
    for i in range(len(x_graph)):
        y_graph[i] = twoLayerReluNet(alpha_m,a,w,b,x_graph[i])

    # Plot the function
    plt.scatter(x, y, c='orange', alpha=0.5)
    plt.plot(x_graph, y_graph)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    plt.xlim(-0.6, 0.6)
    plt.ylim(-0.01, 0.3)
    plt.show()

# Create an interactive slider
interact(plot_values, index=IntSlider(min=0, max=len(a_values)-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='index', max=199999), Output()), _dom_classes=('widget-in…

<function __main__.plot_values(index)>

Code Cell generating min and max y values over time

In [ ]:
# Define the function to plot the values at a given index
def get_values(index):
    a = a_values[index]
    w = w_values[index]
    b = b_values[index]
    
    y_graph = np.zeros((int)(400))
    # Compute the corresponding y values
    for i in range(len(x_graph)):
        y_graph[i] = twoLayerReluNet(alpha_m,a,w,b,x_graph[i])

    return np.min(y_graph), np.max(y_graph)


x_graph = np.linspace(-0.5, 0.5, 400)
# Initialize an array to store y values
min_y_values = np.zeros((int)(n_steps/1000))
max_y_values = np.zeros((int)(n_steps/1000))
for n in range((int)(n_steps/1000)):
    min_y_values[n], max_y_values[n] = get_values(n*1000)

# Create an array of indices
indices = np.arange(n_steps/1000)

# Plot the min and max values
plt.figure(figsize=(10, 6))
plt.plot(indices, min_y_values, color='blue', label='Min y values')
plt.plot(indices, max_y_values, color='red', label='Max y values')
plt.xscale('log')
plt.xlabel('Index x1000')
plt.ylabel('y values')
plt.title('Min and Max y values at index')
plt.legend()
plt.grid(True)
plt.show()